In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split

In [ ]:
df = pd.read_csv('uber.csv')
df.info()

In [ ]:
df.shape

In [ ]:
df.head()

In [ ]:
df.isnull()

In [ ]:
df.drop(columns=["Unnamed: 0", "key"], inplace=True)
df.head()

In [ ]:
df.isnull().sum()

In [ ]:
df['dropoff_latitude'] = df['dropoff_latitude'].fillna(value=df['dropoff_latitude'].mean())
df['dropoff_longitude'] = df['dropoff_longitude'].fillna(value=df['dropoff_longitude'].median())


In [ ]:
df.dtypes

In [ ]:
df.pickup_datetime = pd.to_datetime(df.pickup_datetime)
df.dtypes

In [13]:
df = df.assign(hour = df.pickup_datetime.dt.hour,
               day = df.pickup_datetime.dt.day,
               month = df.pickup_datetime.dt.month,
               year = df.pickup_datetime.dt.year,
               dayofweek = df.pickup_datetime.dt.dayofweek)

In [ ]:
df

In [ ]:
df = df.drop(["pickup_datetime"], axis =1)
df

In [ ]:
from math import *
    
def distance_formula(longitude1, latitude1, longitude2, latitude2):
    travel_dist = []
    
    for pos in range (len(longitude1)):
        lon1, lan1, lon2, lan2 = map(radians, [longitude1[pos], latitude1[pos], longitude2[pos], latitude2[pos]])
        dist_lon = lon2 - lon1
        dist_lan = lan2 - lan1
        
        a = sin(dist_lan/2)**2 + cos(lan1) * cos(lan2) * sin(dist_lon/2)**2
        
        c = 2 * asin(sqrt(a)) * 6371 
        travel_dist.append(c)
            
    return  travel_dist

In [17]:
df['dist_travel_km'] = distance_formula(df.pickup_longitude.to_numpy(), df.pickup_latitude.to_numpy(), df.dropoff_longitude.to_numpy(), df.dropoff_latitude.to_numpy())

In [ ]:
df.plot(kind = "box",subplots = True,layout = (6,2),figsize=(15,20)) #Boxplot to check the outliers
plt.show()

In [ ]:
def remove_outlier(df1 , col):
    Q1 = df1[col].quantile(0.25)
    Q3 = df1[col].quantile(0.75)
    IQR = Q3 - Q1
    lower_whisker = Q1-1.5*IQR
    upper_whisker = Q3+1.5*IQR
    df[col] = np.clip(df1[col] , lower_whisker , upper_whisker)
    return df1

def treat_outliers_all(df1 , col_list):
    for c in col_list:
        df1 = remove_outlier(df , c)
    return df1

In [20]:
df = treat_outliers_all(df , df.iloc[: , 0::])

In [ ]:
df.plot(kind = "box",subplots = True,layout = (7,2),figsize=(15,20)) 
plt.show()

In [ ]:
corr = df.corr() 
corr

In [ ]:
fig,axis = plt.subplots(figsize = (10,6))
sns.heatmap(df.corr(),annot = True) 

In [ ]:
df_x = df[['pickup_longitude','pickup_latitude','dropoff_longitude','dropoff_latitude','passenger_count','hour','day','month','year','dayofweek','dist_travel_km']]
df_y = df['fare_amount']

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(df_x, df_y, test_size=0.2, random_state=1)

In [ ]:
df

In [ ]:
from sklearn.linear_model import LinearRegression

reg = LinearRegression()

reg.fit(x_train, y_train)

In [ ]:
y_pred_lin = reg.predict(x_test)
print(y_pred_lin)

In [ ]:
from sklearn.ensemble import RandomForestRegressor

rf = RandomForestRegressor(n_estimators=100)
rf.fit(x_train,y_train)

In [ ]:
y_pred_rf = rf.predict(x_test)
print(y_pred_rf)

In [ ]:
cols = ['Model', 'RMSE', 'R-Squared']

result_tabulation = pd.DataFrame(columns = cols)

In [ ]:
from sklearn import metrics 
from sklearn.metrics import r2_score 

reg_RMSE = np.sqrt(metrics.mean_squared_error(y_test, y_pred_lin))
reg_squared = r2_score(y_test, y_pred_lin)

full_metrics = pd.Series({
    'Model': "Linear Regression",
    'RMSE': reg_RMSE,
    'R-Squared': reg_squared
})

new_entry = pd.DataFrame([full_metrics])

new_entry = new_entry.dropna(axis=1, how='all')

result_tabulation = pd.concat([result_tabulation, new_entry], ignore_index=True)

print(result_tabulation)

In [ ]:
rf_RMSE = np.sqrt(metrics.mean_squared_error(y_test, y_pred_rf))
rf_squared = r2_score(y_test, y_pred_rf)

full_metrics = pd.Series({'Model': "Random Forest", 'RMSE': rf_RMSE, 'R-Squared': rf_squared})

new_entry = pd.DataFrame([full_metrics])

new_entry = new_entry.dropna(axis=1, how='all')

result_tabulation = pd.concat([result_tabulation, new_entry], ignore_index=True)

print(result_tabulation)